In [30]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os

import random
from datetime import datetime
from time import time
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from box import Box

import warnings

warnings.filterwarnings(action='ignore')

# 1. 학습 설정

In [35]:
config = {
    'data_path' : "/opt/ml/input/data/train" , # 데이터 경로
    
    'submission_path' : "../submission",
    'submission_name' : 'NGCF_submission.csv', 

    'model_path' : "../model", # 모델 저장 경로
    'model_name' : 'NGCF_v1.pt',

    'num_epochs' : 50,
    "reg" : 1e-5,
    'lr' : 0.0001,
    "emb_dim" : 512,
    "layers" : [512, 512],
    'batch_size' : 1024,
    "node_dropout" : 0.2,
    "mess_dropout" : 0.2,

    'valid_samples' : 10, # 검증에 사용할 sample 수
    'seed' : 22,
    'n_batch' : 30,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

In [19]:
if not os.path.isdir(config.model_path):
    os.mkdir(config.model_path)

In [20]:
if not os.path.isdir(config.submission_path):
    os.mkdir(config.submission_path)

# 2. 데이터 전처리

In [21]:
class MakeGraphDataSet():
    """
    GraphDataSet 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, 'train_ratings.csv'))

        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('item')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('user')
        self.num_item, self.num_user = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['item'].apply(lambda x : self.item_encoder[x])
        self.df['user_idx'] = self.df['user'].apply(lambda x : self.user_encoder[x])
        
        self.exist_users = [i for i in range(self.num_user)]
        self.exist_items = [i for i in range(self.num_item)]
        self.user_train, self.user_valid = self.generate_sequence_data()
        self.R_train, self.R_valid, self.R_total = self.generate_dok_matrix()
        self.ngcf_adj_matrix = self.generate_ngcf_adj_matrix()
        self.n_train = len(self.R_train)
        self.batch_size = self.config.batch_size

    def generate_encoder_decoder(self, col : str) -> dict:
        """
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Returns:
            dict: 생성된 user encoder, decoder
        """

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id

        return encoder, decoder
    
    def generate_sequence_data(self) -> dict:
        """
        sequence_data 생성

        Returns:
            dict: train user sequence / valid user sequence
        """
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        for user, item, time in zip(self.df['user_idx'], self.df['item_idx'], self.df['time']):
            users[user].append(item)
        
        for user in users:
            np.random.seed(self.config.seed)

            user_total = users[user]
            valid = np.random.choice(user_total, size = self.config.valid_samples, replace = False).tolist()
            train = list(set(user_total) - set(valid))

            user_train[user] = train
            user_valid[user] = valid # valid_samples 개수 만큼 검증에 활용 (현재 Task와 가장 유사하게)

        return user_train, user_valid
    
    def generate_dok_matrix(self):
        R_train = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        R_valid = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        R_total = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        user_list = self.exist_users
        for user in user_list:
            train_items = self.user_train[user]
            valid_items = self.user_valid[user]
            
            for train_item in train_items:
                R_train[user, train_item] = 1.0
                R_total[user, train_item] = 1.0
            
            for valid_item in valid_items:
                R_valid[user, valid_item] = 1.0
                R_total[user, valid_item] = 1.0
        
        return R_train, R_valid, R_total

    def generate_ngcf_adj_matrix(self):
        adj_mat = sp.dok_matrix((self.num_user + self.num_item, self.num_user + self.num_item), dtype=np.float32)
        adj_mat = adj_mat.tolil() # to_list
        R = self.R_train.tolil()

        adj_mat[:self.num_user, self.num_user:] = R
        adj_mat[self.num_user:, :self.num_user] = R.T
        adj_mat = adj_mat.todok() # to_dok_matrix

        def normalized_adj_single(adj):
            rowsum = np.array(adj.sum(1))
            d_inv = np.power(rowsum, -.5).flatten()  
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)
            norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv)

            return norm_adj.tocoo()

        ngcf_adj_matrix = normalized_adj_single(adj_mat)
        return ngcf_adj_matrix.tocsr()

    def sampling(self):
        users = random.sample(self.exist_users, self.config.batch_size)

        def sample_pos_items_for_u(u, num):
            pos_items = self.user_train[u]
            pos_batch = random.sample(pos_items, num)
            return pos_batch
        
        def sample_neg_items_for_u(u, num):
            neg_items = list(set(self.exist_items) - set(self.user_train[u]))
            neg_batch = random.sample(neg_items, num)
            return neg_batch
        
        pos_items, neg_items = [], []
        for user in users:
            pos_items += sample_pos_items_for_u(user, 1)
            neg_items += sample_neg_items_for_u(user, 1)
        
        return users, pos_items, neg_items

    def get_train_valid_data(self):
        return self.user_train, self.user_valid

    def get_R_data(self):
        return self.R_train, self.R_valid, self.R_total

    def get_ngcf_adj_matrix_data(self):
        return self.ngcf_adj_matrix

# 3. 모델

In [22]:
class NGCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim, layers, reg, node_dropout, mess_dropout, adj_mtx):
        super().__init__()

        # initialize Class attributes
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.l_matrix = adj_mtx
        self.l_plus_i_matrix = adj_mtx + sp.eye(adj_mtx.shape[0])
        self.reg = reg
        self.layers = layers
        self.n_layers = len(self.layers)
        self.node_dropout = node_dropout
        self.mess_dropout = mess_dropout

        # Initialize weights
        self.weight_dict = self._init_weights()
        print("Weights initialized.")

        # Create Matrix 'L+I', PyTorch sparse tensor of SP adjacency_mtx
        self.L_plus_I = self._convert_sp_mat_to_sp_tensor(self.l_plus_i_matrix)
        self.L = self._convert_sp_mat_to_sp_tensor(self.l_matrix)

    # initialize weights
    def _init_weights(self):
        print("Initializing weights...")
        weight_dict = nn.ParameterDict()

        initializer = torch.nn.init.xavier_uniform_
        
        weight_dict['user_embedding'] = nn.Parameter(initializer(torch.empty(self.n_users, self.emb_dim).to(device)))
        weight_dict['item_embedding'] = nn.Parameter(initializer(torch.empty(self.n_items, self.emb_dim).to(device)))

        weight_size_list = [self.emb_dim] + self.layers

        for k in range(self.n_layers):
            weight_dict['W_one_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_one_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
            
            weight_dict['W_two_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_two_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
           
        return weight_dict

    # convert sparse matrix into sparse PyTorch tensor
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res

    # apply node_dropout
    def _droupout_sparse(self, X):
        """
        Drop individual locations in X
        
        Arguments:
        ---------
        X = adjacency matrix (PyTorch sparse tensor)
        dropout = fraction of nodes to drop
        noise_shape = number of non non-zero entries of X
        """
        node_dropout_mask = ((self.node_dropout) + torch.rand(X._nnz())).floor().bool().to(device)
        i = X.coalesce().indices()
        v = X.coalesce()._values()
        i[:,node_dropout_mask] = 0
        v[node_dropout_mask] = 0
        X_dropout = torch.sparse.FloatTensor(i, v, X.shape).to(X.device)

        return  X_dropout.mul(1/(1-self.node_dropout))

    def forward(self, u, i, j):
        """
        Computes the forward pass
        
        Arguments:
        ---------
        u = user
        i = positive item (user interacted with item)
        j = negative item (user did not interact with item)
        """
        # apply drop-out mask
        L_plus_I_hat = self._droupout_sparse(self.L_plus_I) if self.node_dropout > 0 else self.L_plus_I
        L_hat = self._droupout_sparse(self.L) if self.node_dropout > 0 else self.L
        
        # 논문 수식 (1)
        ego_embeddings = torch.cat([self.weight_dict['user_embedding'], self.weight_dict['item_embedding']], 0)

        final_embeddings = [ego_embeddings]

        # forward pass for 'n' propagation layers
        for k in range(self.n_layers):
            
            
            ########## Fill below ###########
            ### 논문 수식 (7) ###
            
            # (L+I)E
            side_L_plus_I_embeddings = torch.sparse.mm(L_plus_I_hat, final_embeddings[k]) #힌트 : use torch.sparse.mm 
            
            # (L+I)EW_1 + b_1
            simple_embeddings = torch.matmul(side_L_plus_I_embeddings, self.weight_dict['W_one_%d' % k]) + self.weight_dict['b_one_%d' % k] #힌트 : use torch.matmul, self.weight_dict['W_one_%d' % k], self.weight_dict['b_one_%d' % k]
            
            # LE
            side_L_embeddings = torch.sparse.mm(L_hat, final_embeddings[k]) #힌트 : use torch.sparse.mm                                
            
            # LEE
            interaction_embeddings = torch.mul(side_L_embeddings, final_embeddings[k]) #힌트 : use torch.mul
                                             
            # LEEW_2 + b_2
            interaction_embeddings = torch.matmul(interaction_embeddings, self.weight_dict['W_two_%d' % k]) + self.weight_dict['b_two_%d' % k]  #힌트 : use torch.matmul, self.weight_dict['W_two_%d' % k], self.weight_dict['b_two_%d' % k]

            # non-linear activation 
            ego_embeddings =  F.leaky_relu(simple_embeddings + interaction_embeddings) #힌트: use simple_embeddings, interaction_embeddings
            
            ########## Fill above ###########
            
            # add message dropout
            mess_dropout_mask = nn.Dropout(self.mess_dropout)
            ego_embeddings = mess_dropout_mask(ego_embeddings)

            # Perform L2 normalization
            norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)
            
            ### 논문 수식 (9) ###
            final_embeddings.append(norm_embeddings)                                            

        
        final_embeddings = torch.cat(final_embeddings, 1)                           
    
        # back to user/item dimension
        u_final_embeddings, i_final_embeddings = final_embeddings.split([self.n_users, self.n_items], 0)

        self.u_final_embeddings = nn.Parameter(u_final_embeddings)
        self.i_final_embeddings = nn.Parameter(i_final_embeddings)
        
        u_emb = u_final_embeddings[u] # user embeddings
        p_emb = i_final_embeddings[i] # positive item embeddings
        n_emb = i_final_embeddings[j] # negative item embeddings
    
        
        ########## Fill below ###########
        ### 논문 수식 (10) ###
        
        y_ui = torch.sum(torch.mul(u_emb, p_emb), dim = 1) # 힌트 : use torch.mul, sum() method                             
        y_uj = torch.sum(torch.mul(u_emb, n_emb), dim = 1) # 힌트 : use torch.mul, sum() method 
        
        ########## Fill above ########### 
        
        ########## Fill below ########### 
        ### 논문 수식 (11) ###
        
        log_prob = torch.mean(torch.log(torch.sigmoid(y_ui - y_uj))) # 힌트 : use torch.log, torch.sigmoid, mean() method
        bpr_loss = -log_prob        
        if self.reg > 0.:
            l2norm = (torch.sum(u_emb**2)/2. + torch.sum(p_emb**2)/2. + torch.sum(n_emb**2)/2.) / u_emb.shape[0]
            l2reg = self.reg * l2norm # FILL HERE #
            bpr_loss += l2reg
        
        ########## Fill above ###########
        
        return bpr_loss

# 4. 학습 함수

In [32]:
def train(model, make_graph_data_set, optimizer, n_batch):
    model.train()
    loss_val = 0
    for step in range(1, n_batch + 1):
        user, pos, neg = make_graph_data_set.sampling()
        optimizer.zero_grad()
        loss = model(user, pos, neg)
        loss.backward()
        optimizer.step()
        loss_val += loss.item()
    loss_val /= n_batch
    return loss_val

def split_matrix(X, n_splits=10):
    splits = []
    chunk_size = X.shape[0] // n_splits
    for i in range(n_splits):
        start = i * chunk_size
        end = X.shape[0] if i == n_splits - 1 else (i + 1) * chunk_size
        splits.append(X[start:end])
    return splits

def compute_ndcg_k(pred_items, test_items, test_indices, k):
    
    r = (test_items * pred_items).gather(1, test_indices)
    f = torch.from_numpy(np.log2(np.arange(2, k+2))).float().to(device)
    
    dcg = (r[:, :k]/f).sum(1)                                               
    dcg_max = (torch.sort(r, dim=1, descending=True)[0][:, :k]/f).sum(1)   
    ndcg = dcg/dcg_max                                                     
    
    ndcg[torch.isnan(ndcg)] = 0
    return ndcg

def evaluate(u_emb, i_emb, Rtr, Rte, k = 10):

    # split matrices
    ue_splits = split_matrix(u_emb)
    tr_splits = split_matrix(Rtr)
    te_splits = split_matrix(Rte)

    recall_k, ndcg_k= [], []
    # compute results for split matrices
    for ue_f, tr_f, te_f in zip(ue_splits, tr_splits, te_splits):

        scores = torch.mm(ue_f, i_emb.t())

        test_items = torch.from_numpy(te_f.todense()).float().to(device)
        non_train_items = torch.from_numpy(1-(tr_f.todense())).float().to(device)
        scores = scores * non_train_items

        _, test_indices = torch.topk(scores, dim=1, k=k)
        
        pred_items = torch.zeros_like(scores).float()
        pred_items.scatter_(dim=1, index=test_indices, src=torch.ones_like(test_indices).float().to(device))

        topk_preds = torch.zeros_like(scores).float()
        topk_preds.scatter_(dim=1, index=test_indices[:, :k], src=torch.ones_like(test_indices).float())
        
        TP = (test_items * topk_preds).sum(1)                      
        rec = TP/test_items.sum(1)
   
        ndcg = compute_ndcg_k(pred_items, test_items, test_indices, k)

        recall_k.append(rec)
        ndcg_k.append(ndcg)

    return torch.cat(ndcg_k).mean(), torch.cat(recall_k).mean()

# 5. 학습

In [24]:
make_graph_data_set = MakeGraphDataSet(config = config)
ngcf_adj_matrix = make_graph_data_set.get_ngcf_adj_matrix_data()
R_train, R_valid, R_total = make_graph_data_set.get_R_data()

In [42]:
model = NGCF(
    n_users = make_graph_data_set.num_user,
    n_items = make_graph_data_set.num_item,
    emb_dim = config.emb_dim,
    layers = config.layers,
    reg = config.reg,
    node_dropout = config.node_dropout,
    mess_dropout = config.mess_dropout,
    adj_mtx = ngcf_adj_matrix,
    ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

Initializing weights...
Weights initialized.


In [43]:
best_hit = 0
for epoch in range(1, config.num_epochs + 1):
    tbar = tqdm(range(1))
    for _ in tbar:
        train_loss = train(
            model = model, 
            make_graph_data_set = make_graph_data_set, 
            optimizer = optimizer,
            n_batch = config.n_batch,
            )
        with torch.no_grad():
            ndcg, hit = evaluate(
                u_emb = model.u_final_embeddings.detach(), 
                i_emb = model.i_final_embeddings.detach(), 
                Rtr = R_train, 
                Rte = R_valid, 
                k = 10,
                )

        if best_hit < hit:
            best_hit = hit
            torch.save(model.state_dict(), os.path.join(config.model_path, config.model_name))

        tbar.set_description(f'Epoch: {epoch:3d}| Train loss: {train_loss:.5f}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

Epoch:   1| Train loss: 0.71478| NDCG@10: 0.00440| HIT@10: 0.00093: 100%|██████████| 1/1 [03:11<00:00, 191.30s/it]
Epoch:   2| Train loss: 0.69208| NDCG@10: 0.01050| HIT@10: 0.00234: 100%|██████████| 1/1 [03:12<00:00, 192.95s/it]
Epoch:   3| Train loss: 0.68492| NDCG@10: 0.01687| HIT@10: 0.00381: 100%|██████████| 1/1 [03:11<00:00, 192.00s/it]
Epoch:   4| Train loss: 0.67697| NDCG@10: 0.02030| HIT@10: 0.00449: 100%|██████████| 1/1 [03:11<00:00, 191.95s/it]
Epoch:   5| Train loss: 0.62087| NDCG@10: 0.02803| HIT@10: 0.00642: 100%|██████████| 1/1 [03:10<00:00, 190.85s/it]
Epoch:   6| Train loss: 0.51995| NDCG@10: 0.03267| HIT@10: 0.00740: 100%|██████████| 1/1 [03:09<00:00, 189.69s/it]
Epoch:   7| Train loss: 0.49010| NDCG@10: 0.03495| HIT@10: 0.00785: 100%|██████████| 1/1 [03:12<00:00, 192.71s/it]
Epoch:   8| Train loss: 0.47771| NDCG@10: 0.04032| HIT@10: 0.00904: 100%|██████████| 1/1 [03:12<00:00, 192.35s/it]
Epoch:   9| Train loss: 0.47207| NDCG@10: 0.04869| HIT@10: 0.01105: 100%|███████

```
Epoch:   1| Train loss: 0.71478| NDCG@10: 0.00440| HIT@10: 0.00093: 100%|██████████| 1/1 [03:11<00:00, 191.30s/it]
Epoch:   2| Train loss: 0.69208| NDCG@10: 0.01050| HIT@10: 0.00234: 100%|██████████| 1/1 [03:12<00:00, 192.95s/it]
Epoch:   3| Train loss: 0.68492| NDCG@10: 0.01687| HIT@10: 0.00381: 100%|██████████| 1/1 [03:11<00:00, 192.00s/it]
Epoch:   4| Train loss: 0.67697| NDCG@10: 0.02030| HIT@10: 0.00449: 100%|██████████| 1/1 [03:11<00:00, 191.95s/it]
Epoch:   5| Train loss: 0.62087| NDCG@10: 0.02803| HIT@10: 0.00642: 100%|██████████| 1/1 [03:10<00:00, 190.85s/it]
Epoch:   6| Train loss: 0.51995| NDCG@10: 0.03267| HIT@10: 0.00740: 100%|██████████| 1/1 [03:09<00:00, 189.69s/it]
Epoch:   7| Train loss: 0.49010| NDCG@10: 0.03495| HIT@10: 0.00785: 100%|██████████| 1/1 [03:12<00:00, 192.71s/it]
Epoch:   8| Train loss: 0.47771| NDCG@10: 0.04032| HIT@10: 0.00904: 100%|██████████| 1/1 [03:12<00:00, 192.35s/it]
Epoch:   9| Train loss: 0.47207| NDCG@10: 0.04869| HIT@10: 0.01105: 100%|██████████| 1/1 [03:12<00:00, 192.60s/it]
Epoch:  10| Train loss: 0.46729| NDCG@10: 0.04227| HIT@10: 0.00970: 100%|██████████| 1/1 [03:08<00:00, 188.64s/it]
Epoch:  11| Train loss: 0.46221| NDCG@10: 0.04642| HIT@10: 0.01054: 100%|██████████| 1/1 [03:09<00:00, 189.42s/it]
Epoch:  12| Train loss: 0.45805| NDCG@10: 0.05219| HIT@10: 0.01192: 100%|██████████| 1/1 [03:11<00:00, 191.62s/it]
Epoch:  13| Train loss: 0.45512| NDCG@10: 0.03641| HIT@10: 0.00845: 100%|██████████| 1/1 [03:09<00:00, 189.67s/it]
Epoch:  14| Train loss: 0.45052| NDCG@10: 0.04267| HIT@10: 0.00991: 100%|██████████| 1/1 [03:10<00:00, 190.13s/it]
Epoch:  15| Train loss: 0.44485| NDCG@10: 0.04894| HIT@10: 0.01137: 100%|██████████| 1/1 [03:09<00:00, 189.51s/it]
Epoch:  16| Train loss: 0.43992| NDCG@10: 0.05674| HIT@10: 0.01287: 100%|██████████| 1/1 [03:12<00:00, 192.47s/it]
Epoch:  17| Train loss: 0.43133| NDCG@10: 0.06884| HIT@10: 0.01587: 100%|██████████| 1/1 [03:10<00:00, 190.51s/it]
Epoch:  18| Train loss: 0.41281| NDCG@10: 0.07942| HIT@10: 0.01836: 100%|██████████| 1/1 [03:11<00:00, 191.57s/it]
Epoch:  19| Train loss: 0.39955| NDCG@10: 0.08121| HIT@10: 0.01882: 100%|██████████| 1/1 [03:11<00:00, 191.62s/it]
Epoch:  20| Train loss: 0.39348| NDCG@10: 0.08293| HIT@10: 0.01955: 100%|██████████| 1/1 [03:11<00:00, 191.57s/it]
Epoch:  21| Train loss: 0.38870| NDCG@10: 0.09752| HIT@10: 0.02268: 100%|██████████| 1/1 [03:11<00:00, 192.00s/it]
Epoch:  22| Train loss: 0.38452| NDCG@10: 0.08017| HIT@10: 0.01882: 100%|██████████| 1/1 [03:09<00:00, 189.95s/it]
Epoch:  23| Train loss: 0.38057| NDCG@10: 0.10778| HIT@10: 0.02632: 100%|██████████| 1/1 [03:10<00:00, 190.45s/it]
Epoch:  24| Train loss: 0.37645| NDCG@10: 0.10301| HIT@10: 0.02486: 100%|██████████| 1/1 [03:08<00:00, 188.79s/it]
Epoch:  25| Train loss: 0.37414| NDCG@10: 0.15097| HIT@10: 0.03728: 100%|██████████| 1/1 [03:11<00:00, 191.23s/it]
Epoch:  26| Train loss: 0.37016| NDCG@10: 0.13321| HIT@10: 0.03376: 100%|██████████| 1/1 [03:08<00:00, 188.33s/it]
Epoch:  27| Train loss: 0.36819| NDCG@10: 0.14423| HIT@10: 0.03754: 100%|██████████| 1/1 [03:11<00:00, 191.36s/it]
Epoch:  28| Train loss: 0.36562| NDCG@10: 0.16245| HIT@10: 0.04175: 100%|██████████| 1/1 [03:10<00:00, 190.80s/it]
Epoch:  29| Train loss: 0.36273| NDCG@10: 0.16760| HIT@10: 0.04472: 100%|██████████| 1/1 [03:11<00:00, 191.20s/it]
Epoch:  30| Train loss: 0.36049| NDCG@10: 0.17515| HIT@10: 0.04572: 100%|██████████| 1/1 [03:11<00:00, 191.48s/it]
Epoch:  31| Train loss: 0.35784| NDCG@10: 0.18736| HIT@10: 0.05031: 100%|██████████| 1/1 [03:11<00:00, 191.82s/it]
Epoch:  32| Train loss: 0.35471| NDCG@10: 0.18383| HIT@10: 0.04919: 100%|██████████| 1/1 [03:10<00:00, 190.34s/it]
Epoch:  33| Train loss: 0.35317| NDCG@10: 0.19398| HIT@10: 0.05257: 100%|██████████| 1/1 [03:10<00:00, 190.78s/it]
Epoch:  34| Train loss: 0.35102| NDCG@10: 0.19721| HIT@10: 0.05356: 100%|██████████| 1/1 [03:12<00:00, 192.39s/it]
Epoch:  35| Train loss: 0.34800| NDCG@10: 0.21801| HIT@10: 0.06023: 100%|██████████| 1/1 [03:11<00:00, 191.69s/it]
Epoch:  36| Train loss: 0.34548| NDCG@10: 0.20902| HIT@10: 0.05692: 100%|██████████| 1/1 [03:09<00:00, 189.25s/it]
Epoch:  37| Train loss: 0.34382| NDCG@10: 0.20885| HIT@10: 0.05883: 100%|██████████| 1/1 [03:09<00:00, 189.57s/it]
Epoch:  38| Train loss: 0.34078| NDCG@10: 0.21918| HIT@10: 0.05975: 100%|██████████| 1/1 [03:09<00:00, 189.33s/it]
Epoch:  39| Train loss: 0.33765| NDCG@10: 0.22726| HIT@10: 0.06327: 100%|██████████| 1/1 [03:11<00:00, 191.10s/it]
Epoch:  40| Train loss: 0.33500| NDCG@10: 0.23527| HIT@10: 0.06483: 100%|██████████| 1/1 [03:11<00:00, 191.28s/it]
Epoch:  41| Train loss: 0.33249| NDCG@10: 0.23729| HIT@10: 0.06484: 100%|██████████| 1/1 [03:12<00:00, 192.68s/it]
Epoch:  42| Train loss: 0.33103| NDCG@10: 0.24932| HIT@10: 0.06783: 100%|██████████| 1/1 [03:11<00:00, 191.41s/it]
Epoch:  43| Train loss: 0.32882| NDCG@10: 0.24583| HIT@10: 0.06680: 100%|██████████| 1/1 [03:10<00:00, 190.75s/it]
Epoch:  44| Train loss: 0.32542| NDCG@10: 0.24051| HIT@10: 0.06725: 100%|██████████| 1/1 [03:10<00:00, 190.48s/it]
Epoch:  45| Train loss: 0.32313| NDCG@10: 0.25097| HIT@10: 0.06803: 100%|██████████| 1/1 [03:11<00:00, 191.53s/it]
Epoch:  46| Train loss: 0.31893| NDCG@10: 0.24085| HIT@10: 0.06794: 100%|██████████| 1/1 [03:08<00:00, 188.04s/it]
Epoch:  47| Train loss: 0.31496| NDCG@10: 0.24480| HIT@10: 0.06904: 100%|██████████| 1/1 [03:09<00:00, 189.67s/it]
Epoch:  48| Train loss: 0.31329| NDCG@10: 0.26454| HIT@10: 0.07297: 100%|██████████| 1/1 [03:11<00:00, 191.90s/it]
Epoch:  49| Train loss: 0.31094| NDCG@10: 0.26029| HIT@10: 0.07303: 100%|██████████| 1/1 [03:10<00:00, 190.19s/it]
Epoch:  50| Train loss: 0.30915| NDCG@10: 0.26914| HIT@10: 0.07489: 100%|██████████| 1/1 [03:09<00:00, 189.71s/it]
```

# 6. 예측

In [68]:
model.load_state_dict(torch.load(os.path.join(config.model_path, config.model_name)))
submision = []
with torch.no_grad():
    k = 10
    u_emb = model.u_final_embeddings.detach()
    i_emb = model.i_final_embeddings.detach()

    scores = torch.mm(u_emb, i_emb.t())

    test_items = torch.from_numpy(R_total.todense()).float().to(device)
    non_train_items = torch.from_numpy(1-(R_total.todense())).float().to(device)
    scores = scores * non_train_items

    _, test_indices = torch.topk(scores, dim=1, k=k)

    pred_items = torch.zeros_like(scores).float()
    pred_items.scatter_(dim=1, index=test_indices, src=torch.ones_like(test_indices).float().to(device))
    pred_items = pred_items.argsort(dim = 1)
    for user, item_list in enumerate(pred_items):
        item_list = item_list[-10:].cpu().numpy().tolist()
        for item in item_list:
            submision.append(
                {   
                    'user' : make_graph_data_set.user_decoder[user],
                    'item' : make_graph_data_set.item_decoder[item],
                }
            )

submision = pd.DataFrame(submision)


In [69]:
submision.to_csv(os.path.join(config.submission_path, config.submission_name), index=False)

In [70]:
submision

,user,item
0,11,1265
1,11,1704
2,11,1193
3,11,3147
4,11,70286
...,...,...
313595,138493,608
313596,138493,8368
313597,138493,1704
313598,138493,50
